In [ ]:
import torch
import numpy as np
from sklearn.metrics import roc_auc_score

In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))])

training_data = datasets.CIFAR10(
    root="data", # root is the path where the train/test data is stored
    train=True, # train specifies training or test dataset
    download=True, # download=True downloads the data from the internet if it’s not available at root
    transform=transform # transform and target_transform specify the feature and label transformations
)

test_data = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=transform
)

trainloader = torch.utils.data.DataLoader(training_data, batch_size=128,
                                          shuffle=True, num_workers=2)

testloader = torch.utils.data.DataLoader(test_data, batch_size=128,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
from torch import nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 3, 3)
        self.conv1_1 = nn.Conv2d(3, 6, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 6, 3)
        self.conv2_2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
      
        x = self.pool(F.relu(self.conv1_1(self.conv1(x))))
        x = self.pool(F.relu(self.conv2_2(self.conv2(x))))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
import torch.optim as optim
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CNN()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(50):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 0:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,     1] loss: 0.001
[1,   101] loss: 0.115
[1,   201] loss: 0.115
[1,   301] loss: 0.115
[2,     1] loss: 0.001
[2,   101] loss: 0.115
[2,   201] loss: 0.115
[2,   301] loss: 0.115
[3,     1] loss: 0.001
[3,   101] loss: 0.115
[3,   201] loss: 0.115
[3,   301] loss: 0.115
[4,     1] loss: 0.001
[4,   101] loss: 0.115
[4,   201] loss: 0.115
[4,   301] loss: 0.114
[5,     1] loss: 0.001
[5,   101] loss: 0.113
[5,   201] loss: 0.111
[5,   301] loss: 0.110
[6,     1] loss: 0.001
[6,   101] loss: 0.108
[6,   201] loss: 0.106
[6,   301] loss: 0.105
[7,     1] loss: 0.001
[7,   101] loss: 0.102
[7,   201] loss: 0.101
[7,   301] loss: 0.100
[8,     1] loss: 0.001
[8,   101] loss: 0.099
[8,   201] loss: 0.097
[8,   301] loss: 0.097
[9,     1] loss: 0.001
[9,   101] loss: 0.096
[9,   201] loss: 0.095
[9,   301] loss: 0.094
[10,     1] loss: 0.001
[10,   101] loss: 0.093
[10,   201] loss: 0.092
[10,   301] loss: 0.092
[11,     1] loss: 0.001
[11,   101] loss: 0.091
[11,   201] loss: 0.090
[11,

In [ ]:
torch.cuda.empty_cache()

vgg11.to(device)


In [ ]:
correct = 0
total = 0

# since we're not training, we don't need to calculate the gradients for our outputs
# vgg11.eval()
# with torch.no_grad():
for data in testloader:
        images, labels = data
        # images, labels = images.to(device), labels.to(device)

        # calculate outputs by running images through the network
        outputs = vgg11(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

In [ ]:
# prepare to count predictions for each class
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        
        outputs = vgg11(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print("Accuracy for class {:5s} is: {:.1f} %".format(classname,
                                                   accuracy))

KeyboardInterrupt: ignored